In [157]:
# Implementation of optimum contraction scheme.
# not general assumes Rank of C to be 3 and rank of X to be 4

include("../../22tn-j_CodeRepository/Tensor.jl")
using LinearAlgebra

function updateoptleft(C::Array,rankC::Int64,B::Array, X::Array{Float64},rankX::Int64,A::Array{Float64})
    rankB = length(size(B)); rankA = length(size(A));
    if rankB != 3
        error("ERR: Input tensor B cannot be used")
    end
    if rankA != 3
        error("ERR: Input tensor A cannot be used")
    end
    Bt = conj(B)
    T = contract(C,rankC,[rankC],A,rankA,[1])
    rankT = rankA+rankC-2 # 2 = number of indices contracted
    T2 = contract(X,rankX,[1,4],T,rankT,[2,3])
    rankT2 = rankT+rankX-4
    Cn = contract(Bt,rankB,[1,2],T2,rankT2,[1,3])
    return Cn
end

updateoptleft (generic function with 1 method)

The order of contraction matters. The order of contraction is governed by the direction given in the figure:

<img src="./contract.png" width="150" height="150">

For instance while contracting $C$ and $A$ we need do $contract(C,rankC,[indicesC],A,rankA,[indicesA])$ instead if we do $contract(A,rankA,[indicesA],C,rankC,[indicesC])$ there will be problem in the dimensions.

Similarly all contraction should be done according to the direction. But in this contraction scheme there is a problem, If we contract $C$ and $A$ then, the resultant tensor cannot be contracted with $X$ directly.The two indices to be contracted with X will be of opposite directions.

<img src="./contract2.png" width="200" height="200">

I am not able to find out how to deal with such situations. Thus the answer I get varies from the result given in the solution

In [160]:
# my implementation of the solution given
# not general assumes Rank of C to be 3 and rank of X to be 4

include("../../22tn-j_CodeRepository/Tensor.jl")
using LinearAlgebra

function myupdateleft(C::Array,rankC::Int64,B::Array, X::Array{Float64},rankX::Int64,A::Array{Float64})
    rankB = length(size(B)); rankA = length(size(A));
    if rankB != 3
        error("ERR: Input tensor B cannot be used")
    end
    if rankA != 3
        error("ERR: Input tensor A cannot be used")
    end
    Bt = conj(B)
    T = contract(X,rankX,rankX,A,rankA,2) # direction of arrow matters
    rankT = rankA+rankX-2 #2 = number of indices contracted
    T1 = contract(C,rankC,[2,rankC],T,rankX+1,[1,4])
    rankT1 = rankT+rankC-4
    Cn = contract(Bt,rankB,[1,2],T1,rankC+rankX-3,[1,2])
    return Cn
end

myupdateleft (generic function with 1 method)

In [169]:
# Setting up the dimensions

d = 4;
w = 6;
Dset = (100:20:300);


In [170]:
## Intialising A,B,C,X
D = Dset[2];
A = rand(D,d,D);
B = rand(D,d,D);
C = rand(D,w,D);
X = rand(w,d,w,d);

In [161]:
include("../../22tn-j_CodeRepository/Tensor.jl")
ans1 = updateLeft(C,3,B,X,4,A);

In [ ]:
ans2 = myupdateleft(C,3,B,X,4,A);
ans3 = updateoptleft(C,3,B,X,4,A);

In [163]:
ans1==ans2

true

In [164]:
ans1==ans3

false

In [165]:
ans2 == ans3

false

In [172]:
num_avg = 100;

timeC = zeros(size(Dset))
for idx in (1:length(Dset))
    D = Dset[idx];
    for i in (1:num_avg)
        A = rand(D,d,D);
        B = rand(D,d,D);
        C = rand(D,w,D);
        X = rand(w,d,w,d);

        cput = @elapsed begin
        updateLeft(C,3,B,X,4,A)
        end
        timeC[idx] += cput
    end
end

timeC = timeC/num_avg;


In [174]:
import Pkg; Pkg.add("Plots")

  Installing known registries into `C:\Users\navan\.julia`


    Updating registry at `C:\Users\navan\.julia\registries\General.toml`
   Resolving package versions...


   Installed LERC_jll ───────────────────── v3.0.0+1


   Installed JpegTurbo_jll ──────────────── v2.1.91+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed GR_jll ─────────────────────── v0.72.10+0


   Installed Opus_jll ───────────────────── v1.3.2+0


...

   Installed Requires ───────────────────── v1.3.0
   Installed Xorg_libX11_jll ────────────── v1.8.6+0
   Installed Xorg_libXdmcp_jll ──────────── v1.1.4+0


   Installed Unzip ──────────────────────── v0.2.0


   Installed LaTeXStrings ───────────────── v1.3.1
   Installed UnitfulLatexify ────────────── v1.6.3


   Installed SortingAlgorithms ──────────── v1.2.0


   Installed GR ─────────────────────────── v0.72.10


In [173]:
using Plots

plot(Dset,time_C, yaxis=:log, seriestype=:scatter, label = "Time to get C",
    legend=:bottomright, xlabel = "D", ylabel = "Average CPU Time (s)")
plot!(Dset,time_C[end]*(Dset/maximum(Dset)).^3, label = "O(D^3)")

ArgumentError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.